In [10]:
import pdfplumber
import pandas as pd

pdf_path = r"C:\Users\why19\Desktop\河北省环京津地区以水灭火等关键能力建设项目可行性研究报告（审定稿）(2).pdf"

# 函数：从表格上方提取表格名称
def extract_table_name(page, table_bbox):
    # 从表格上方提取文本区域
    top_area = (table_bbox[0], table_bbox[1] - 50, table_bbox[2], table_bbox[1])
    text = page.within_bbox(top_area).extract_text()
    return text.split('\n')[-1] if text else "Unknown Table Name"

tables_dict = {}

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        for table_index, table in enumerate(page.extract_tables(), start=1):
            # 使用第一行作为表头创建 DataFrame
            df = pd.DataFrame(table[1:], columns=table[0])
            
            # 提取表格的边界框
            table_bbox = page.find_tables()[table_index - 1].bbox

            # 提取表格名称
            table_name = extract_table_name(page, table_bbox)

            # 添加表格名称列
            df["Table Name"] = table_name
            
            # 将 DataFrame 存储在字典中
            tables_dict[table_name] = df

# 示例：显示提取的表格
for table_name, table_df in tables_dict.items():
    print(f"Table: {table_name}")
    print(table_df)
    print("\n")

# 将表格保存为单独的 CSV 文件，使用 `utf-8-sig` 编码以防止 Excel 打开时出现乱码
for table_name, table_df in tables_dict.items():
    csv_file = f"{table_name}.csv"
    table_df.to_csv(csv_file, index=False, encoding='utf-8-sig')

Table: 1.1.9 主要经济指标表
     序号                建设项目 工程数量 单位   指标(元)     Table Name
0     一                设备建设                  1.1.9 主要经济指标表
1   （一）              基础设施建设                  1.1.9 主要经济指标表
2     1  蓄水池（储水罐）（含土建工程及安装）  264  座  160000  1.1.9 主要经济指标表
3     2    森林消防水鹤（含土建工程及安装）   11  座   40000  1.1.9 主要经济指标表
4   （二）             防灭火装备建设                  1.1.9 主要经济指标表
5     1                专用车辆                  1.1.9 主要经济指标表
6   1.1              小型消防水车   36  辆  320000  1.1.9 主要经济指标表
7   1.2                 机具车   13  辆  200000  1.1.9 主要经济指标表
8   1.3          全地形森林消防泵浦车    3  辆  180000  1.1.9 主要经济指标表
9     2              以水灭火装备                  1.1.9 主要经济指标表
10  2.1               风力灭火机  442  台    6000  1.1.9 主要经济指标表
11  2.2             涡轮风水灭火机  230  台   68000  1.1.9 主要经济指标表
12  2.3            高压细水雾灭火机  414  台   15000  1.1.9 主要经济指标表
13  2.4           高压便携森林消防泵  137  台   65000  1.1.9 主要经济指标表
14  2.5               便携式水泵   89  台   40000  1.1.9 主要经济指标表
15  2.6                  油锯  390  台